In [1]:
from typing import Dict, List
from autogen import ConversableAgent
import sys
import autogen
import os
from dotenv import load_dotenv

from main import fetch_restaurant_data, calculate_overall_score, get_data_fetch_agent_prompt, analyze_reviews

/Users/filip/code/cs-294/lab01/main.py:150: SyntaxWarning: invalid escape sequence '\d'
  score_message = """


In [20]:
restaurant_name = "Subway"

restaurant_data = fetch_restaurant_data(restaurant_name)
print(restaurant_data)

food_scores, service_scores = analyze_reviews(restaurant_data[restaurant_name])
print(food_scores)
print(service_scores)

overall_score = calculate_overall_score(restaurant_name, food_scores, service_scores)
print(overall_score)

{'Subway': ['Subway offers a good selection of fresh ingredients for their sandwiches. The customer service was enjoyable, with friendly staff who were attentive to customization requests.', 'The sandwiches were good and freshly made. The staff was friendly and efficient, making it an enjoyable experience overall.', 'I had a good experience at Subway. The sandwiches were satisfying and fresh, and the staff was friendly and efficient.', 'Subway offers a good selection of fresh ingredients for their sandwiches. The staff was friendly and efficient, making for an enjoyable lunch experience.', 'Both the food and service were good at Subway. The sandwich was enjoyable and freshly made, while the staff was friendly and efficient.', 'Subway offers a good selection of fresh ingredients for their sandwiches. The staff was friendly and efficient, making for an enjoyable lunch experience.', 'Subway offers a good variety of fresh ingredients for their sandwiches. The staff was friendly and efficie

In [ ]:
# Load environment variables from the .env file
load_dotenv()

# Do not modify the signature of the "main" function.
def main(user_query: str):
    entrypoint_agent_system_message = """
        You are an entrypoint agent responsible for overseeing and coordinating conversations between other agents. 
        Your task is to manage queries related to restaurant reviews. First, analyze the user's query and extract the 
        relevant restaurant name. Then, interact with the data fetch agent to retrieve the appropriate reviews for that 
        restaurant. Once you have the reviews, pass them on for further processing and summarization.
        """
    
    # Example LLM config for the entrypoint agent
    llm_config = {"config_list": [{"model": "gpt-4o-mini", "api_key": os.environ.get("OPENAI_API_KEY")}]}
    
    # The main entrypoint/supervisor agent
    entrypoint_agent = ConversableAgent("entrypoint_agent", system_message=entrypoint_agent_system_message, llm_config=llm_config)

    # Agents for the different tasks
    fetch_message = """
    You are responsible for fetching reviews for the requested restaurant. 
    Please find all reviews associated with the restaurant mentioned in the user query.
    """

    review_message = """
    Your task is to analyze the reviews and extract two scores for each review:
    - Food score based on the keywords: awesome, good, average, bad, etc.
    - Customer service score based on keywords: amazing, horrible, unpleasant, etc.
    """

    score_message = """
    Your task is to calculate the overall score based on the food and customer service 
    scores, which is computed as the following: SUM(sqrt(food_scores[i]**2 * customer_service_scores[i]) * 1/(N * sqrt(125)) * 10
    """

    datafetch_agent = ConversableAgent(
        name = "Data_fetch_agent",
        llm_config = llm_config,
        system_message = fetch_message
    )

    review_agent = ConversableAgent(
        name = "Review_analysis_agent",
        llm_config = llm_config,
        system_message = review_message
    )

    scoring_agent = ConversableAgent(
        name = "Scoring_agent",
        llm_config = llm_config,
        system_message = score_message
    )

    entrypoint_agent.register_for_llm(name="fetch_restaurant_data", description="Fetches the reviews for a specific restaurant.")(fetch_restaurant_data)
    entrypoint_agent.register_for_execution(name="fetch_restaurant_data")(fetch_restaurant_data)

    datafetch_agent.register_for_llm(name="fetch_restaurant_data", description="Fetches the reviews for a specific restaurant.")(fetch_restaurant_data)
    datafetch_agent.register_for_execution(name="fetch_restaurant_data")(fetch_restaurant_data)

    review_agent.register_for_llm(name="finding_score", description="Finding restaurants's food_scores and customer_service_scores in the relevant reviews")(analyze_reviews)
    review_agent.register_for_execution(name="finding_score")(analyze_reviews)

    entrypoint_agent.register_for_llm(name="calculate_overall_score", description="Calculate restaurants's overall scores based on food_scores and customer_service_scores")(calculate_overall_score)
    entrypoint_agent.register_for_execution(name="calculate_overall_score")(calculate_overall_score)

    scoring_agent.register_for_llm(name="calculate_overall_score", description="Calculate restaurants's overall scores based on food_scores and customer_service_scores")(calculate_overall_score)
    scoring_agent.register_for_execution(name="calculate_overall_score")(calculate_overall_score)

    # TODO
    # Fill in the argument to `initiate_chats` below, calling the correct agents sequentially.
    # If you decide to use another conversation pattern, feel free to disregard this code.
    task1 = f"Fetch reviews for {user_query}."
    task2 = f"Analyze the fetched reviews and extract food_score and customer_service_score."
    task3 = f"Calculate overall score using the extracted scores."

    prefix1 = entrypoint_agent.initiate_chat(
        datafetch_agent,
        message=task1,
        max_turns=4,
        summary_method="last_msg",
    )

    combined_message1 = f"{prefix1}\n\nNew task: {task2}"

    prefix2 = entrypoint_agent.initiate_chat(
        review_agent,
        message=combined_message1,
        max_turns=2,
        summary_method="last_msg",
    )

    combined_message2 = f"{prefix2}\n\nNew task: {task3}"

    result = entrypoint_agent.initiate_chat(
        scoring_agent,
        message=combined_message2,
        max_turns=1,
        summary_method="last_msg",
    )

    # Uncomment once you initiate the chat with at least one agent.
    # result = entrypoint_agent.initiate_chats([{}])

    return result
    
# DO NOT modify this code below.
if __name__ == "__main__":
    assert len(sys.argv) > 1, "Please ensure you include a query for some restaurant when executing main."
    # main(sys.argv[1])
    # print(fetch_restaurant_data("Krispy Kreme"))
    main("Hur bra är restaurangen Subway överlag?")
    # main("How good is the restaurant Chick-fil-A overall?")
    # main("How good is the restaurant In N Out?")
    # main("What is the overall score for Krispy Kreme?")